In [1]:
import featuretools as ft
import pandas as pd
import numpy as np
import seaborn as sns
import simfin as sf
import math
import datetime as date
from datetime import timedelta  
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
import yfinance as yf
import statsmodels.api as sm


import warnings
warnings.filterwarnings("ignore")

In [3]:
#collect data from api
sf.set_api_key('PLEbHyzQgm98TrvHFejzBwikq1mKiJob ')
sf.set_data_dir('~/simfin_data/')
sf.set_data_dir('C:/Users/suher/Python/Springboard/Capstone 2')
sector_industry = sf.load_industries()
companies = sf.load_companies(market='us')
historical_prices = sf.load_shareprices(variant = 'daily', market = 'us',refresh_days = 0)
price_ratios = sf.load_derived_shareprices(variant='daily', market='us',refresh_days = 0)


sp500_daily = pd.read_csv('sp500daily.csv')
sp500_list = list(pd.read_csv('constituents_csv.csv')['Symbol'])
sp500_daily.Date =  pd.to_datetime(sp500_daily.Date)

hp = historical_prices.reset_index()
hp = hp[hp['Ticker'].isin(sp500_list)]
companies = companies.reset_index()
companies = companies[companies['Ticker'].isin(sp500_list)]

sector_industry.Sector.unique()

sp500_daily.set_index('Date',inplace=True)

sector_industry.reset_index(inplace=True)
comp_sector = pd.merge(companies,sector_industry,how='left',left_on='IndustryId',right_on='IndustryId')

hp_ = pd.merge(hp,comp_sector[['SimFinId','Sector']],how='left',left_on='SimFinId',right_on='SimFinId')

t_hp = hp_[hp_['Sector'] == 'Technology']
f_hp = hp_[hp_['Sector'] == 'Financial Services']

t_close = t_hp.set_index('Date').pivot(columns='Ticker',values='Close')
f_close = f_hp.set_index('Date').pivot(columns='Ticker',values='Close')

Dataset "industries" on disk (27 days old).
- Loading from disk ... Done!
Dataset "us-companies" on disk (27 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (1 days old).
- Downloading ... 51.5%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-derived-shareprices-daily" on disk (1 days old).
- Downloading ... 65.4%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



- Downloading ... 92.7%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Done!


In [4]:
def get_etf(t_list): 
    etf_ret = pd.DataFrame(index = t_close.index[-125:], columns = tech_etf)
    for i in tech_etf: 
        cur = yf.Ticker(i)
        etf_ret[i] = cur.history(period="MAX")[-125:]['Close'].pct_change()[1:]
    return etf_ret

In [5]:
t_close_ret = t_close.pct_change()[1:]
tech_etf = ['IXN','IGM','IGV','XNTK','XLK','XSD','VGT','RYT','XSW']
finance_etf= ['XLF','VFH']
etf_ret = get_etf(tech_etf)
ret = etf_ret.join(t_close_ret).dropna()

In [8]:
x = ret.loc[:,tech_etf]
y = ret.loc[:,'AAPL':]
s_score_tech = calculate_signals(y,x,60)[:-1]
s_score_tech.to_csv('1121s.csv')

2020-11-20 00:00:00
running for AAPL
currently at iteration: 0
currently at iteration: 60
running for ADBE
currently at iteration: 0
currently at iteration: 60
running for ADI
currently at iteration: 0
currently at iteration: 60
running for ADSK
currently at iteration: 0
currently at iteration: 60
running for AKAM
currently at iteration: 0
currently at iteration: 60
running for AMAT
currently at iteration: 0
currently at iteration: 60
running for AMD
currently at iteration: 0
currently at iteration: 60
running for ANET
currently at iteration: 0
currently at iteration: 60
running for ANSS
currently at iteration: 0
currently at iteration: 60
running for APH
currently at iteration: 0
currently at iteration: 60
running for ATVI
currently at iteration: 0
currently at iteration: 60
running for AVGO
currently at iteration: 0
currently at iteration: 60
running for CDNS
currently at iteration: 0
currently at iteration: 60
running for CDW
currently at iteration: 0
currently at iteration: 60
runn

In [9]:
params = {}
for i in s_score_tech.columns: 
    print(i)
    s_score_train = pd.DataFrame(s_score_tech[i]).rename({i:i+"_s"},axis=1)
    s_score_train = s_score_train.join(ret[i])
    params[i] = opt_bayesian(s_score_train)
    

AAPL
-1.0198461144446225 0.7549222709763855
|   iter    |  target   |   d_cut   |   u_cut   |
-------------------------------------------------
|  1        | -0.329    | -0.3661   |  0.08237  |
|  2        | -2.669    | -0.8026   |  0.2185   |
|  3        | -0.1085   | -0.1277   |  0.3691   |
|  4        | -0.329    | -0.3419   |  0.2297   |
|  5        | -2.669    | -1.011    |  0.5322   |
|  6        | -0.1085   | -0.1285   |  0.3868   |
|  7        | -0.1005   |  0.0      |  0.7549   |
|  8        | -0.03514  |  0.0      |  0.0      |
|  9        | -0.094    | -0.3167   |  0.7536   |
|  10       | -0.2617   | -0.1714   |  0.0      |
|  11       | -0.03459  |  0.0      |  0.218    |
|  12       | -0.094    | -0.1632   |  0.7549   |
|  13       | -0.1005   |  0.0      |  0.5272   |
|  14       | -0.094    | -0.2677   |  0.5997   |
|  15       | -0.12     |  0.0      |  0.101    |
|  16       | -0.1079   |  0.0      |  0.3589   |
|  17       | -0.1085   | -0.129    |  0.381    |
|  18 

In [36]:
todays = [] 
for i in params: 
    if type(params[i]) is dict: 
        todays.append([i,params[i]])

In [37]:
todays_signals = pd.DataFrame(todays,columns=['ticker','signals']).set_index('ticker')
todays_signals = todays_signals['signals']

In [38]:
s_today = s_score_tech.iloc[-1]

In [39]:
action_today = pd.Series(index = s_today.index)
action_today.iloc[:] = 'nan'
for i in s_today.index: 
    if i in todays_signals.index: 
        if s_today.loc[i] < todays_signals[i]['d_cut']: 
            print('BUY ' + i)
            action_today[i] = 'BUY'
        elif s_today[i] > todays_signals[i]['u_cut']:
            print('SELL ' + i)
            action_today[i] = 'SELL'
        else: 
            'NO TOUCH'
    else: 
        action_today[i] = 'NaN'

SELL AAPL
SELL ADI
SELL AMAT
BUY ANSS
SELL FB
BUY KLAC
BUY MSI
SELL NOW
SELL WDC


In [40]:
action_today[action_today=="BUY"],action_today[action_today=="SELL"]

(ANSS    BUY
 KLAC    BUY
 MSI     BUY
 dtype: object,
 AAPL    SELL
 ADI     SELL
 AMAT    SELL
 FB      SELL
 NOW     SELL
 WDC     SELL
 dtype: object)

In [7]:
def sim_long_arb(d_cut = -1,u_cut = 1): 
    hold = False;
    cap = 1;
    returns = [];
    transaction_date = []
    for i in s_score_train.iterrows():
        date = i[0]
        current_s = i[1][0]
        current_r = i[1][1]
        if(hold == True): 
            returns.append(current_r)
            if(current_s >= u_cut): 
                hold = False
        elif(hold == False and current_s <= d_cut):
            hold = True
    returns = np.array(returns)
    std = np.std(returns[returns<0])
    if std == 0: 
        return np.mean(returns)
    else: 
        sharpe = np.mean(returns)/ std
        return sharpe

def sim_short_arb(d_cut = -1,u_cut = 1,ret=False): 
    short = False;
    cap = 1;
    returns = [];
    transaction_date = []
    for i in s_score_train.iterrows():
        date = i[0]
        current_s = i[1][0]
        current_r = i[1][1] * -1
        if(short == True): 
            returns.append(current_r)
            if(current_s <= d_cut): 
                short = False
        elif(short == False and current_s >= u_cut):
            short = True
    returns = np.array(returns)
    sharpe = np.mean(returns)/np.std(returns)
    if ret==True: 
        return sharpe,returns
    else:
        return sharpe
    
def regress_pca(stock,pca_factors): 
    pca_f = PCA(n_components = 1)
    pca_f.fit(pca_factors)
    features = pca_f.transform(pca_factors)
    features = pd.DataFrame(features,index=pca_factors.index,columns=['etf_pca1'])
    x_pca = sm.add_constant(features)
    y = stock
    lr = sm.OLS(y,x_pca)
    results = lr.fit()
    alpha = results.params[0]
    beta1 = results.params[1]
    res = y - beta1 * x_pca.iloc[:,1] -alpha
    #beta2 = results.params[2]
    return alpha,beta1,res

def calc_ar1(residual): 
    ar1 = ARIMA(residual,order=(1,0,0)).fit()
    ar1_a = ar1.params[0]
    ar1_b = ar1.params[1]
    ar1_res  = ar1.resid
    return ar1_a,ar1_b,ar1_res

def compute_s_score(a,b,r):
    m = r.mean()
    if b > 0: 
        std_r_eq = r.std()/np.sqrt(2 * b)
    else: 
        std_r_eq =  r.std()/-np.sqrt(2 * abs(b))
    s = (r[-1]- m)/std_r_eq
    return s,std_r_eq,m


def calculate_signals(s,b,window): 
    signal = pd.DataFrame(columns = s.columns, index = s.index[window:])
    print(signal.index[-1])
    for i in range(0,s.shape[1]): 
        print('running for'+ " " + s.columns[i])
        try:
            for j in range(0,s.shape[0]-window+1):
                date = s.index[j + window -1]
                if(j%60 == 0): 
                    print("currently at iteration: "+ str(j))

                stock = s.iloc[j:j+window,i]
                pca_f = b.iloc[j:j+window,:]
                alpha,beta,res = regress_pca(stock,pca_f)
                #beta2 = results.params[2]           
                ar1_a,ar1_b,ar1_res = calc_ar1(res.mean() - res)
                day_s = compute_s_score(ar1_a,ar1_b,res)[0]
                signal.loc[date,s.columns[i]]=day_s

        except: 
            print('error calculating for: ' + s.columns[i])
    return signal
        
from itertools import product
import random
def opt_bayesian(s): 
    smin = s.iloc[:,0].min()
    smax = s.iloc[:,0].max()
    print(smin,smax)
    opt_long = BayesianOptimization(sim_long_arb,{'d_cut':(smin,0),'u_cut':(0,smax)})
    try: 
        opt_long.maximize(n_iter=100)
        return opt_long.max['params']
    except: 
        return -100,-100
        